In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def eda(data):
    df = data.copy()
    df = df[['Region', 'UTC time', 'Local date', 'Hour', 'Local time', 'Time zone',
                         'DF', 'D', 'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC',
                         'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH', 'CO2 Emissions Generated']]
    df['Local date'] = pd.to_datetime(df['Local date'], format='%d%b%Y')
    df['day'] = df['Local date'].dt.day
    df['month'] = df['Local date'].dt.month
    df['year'] = df['Local date'].dt.year
    df['dotw'] = df['Local date'].dt.dayofweek
    df = df[df['year'] >= 2019]
    df['Hour'] = df['Hour'] - 1
    df = df[~((df['day'] == 18) & (df['month'] == 11) & (df['year'] == 2024))]
    df = df[~((df['day'] == 17) & (df['month'] == 11) & (df['year'] == 2024))]
    df['D'] = pd.to_numeric(df['D'].str.replace(',', '', regex=False), errors='coerce')
    return df

In [10]:
california = eda(pd.read_csv('california.csv', low_memory=False))

In [58]:
california['NG: SUN'] = pd.to_numeric(california['NG: SUN'], errors='coerce')
california['NG: WND'] = pd.to_numeric(california['NG: WND'], errors='coerce')

In [70]:
summer_2024 = pd.read_csv('2024_Summer.csv')
summer_2023 = pd.read_csv('2023_Summer.csv')
summer_2022 = pd.read_csv('2022_Summer.csv')
summer_2021 = pd.read_csv('2021_Summer.csv')
summer_2024_cleaned = summer_2024[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN']]
summer_2023_cleaned = summer_2023[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN']]
summer_2022_cleaned = summer_2022[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN']]
summer_2021_cleaned = summer_2021[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN']]

In [74]:
all_summer_data = pd.concat([summer_2024_cleaned, 
                             summer_2023_cleaned, 
                             summer_2022_cleaned, 
                             summer_2021_cleaned], 
                            ignore_index=True)

In [62]:
cal_bydate = california.groupby('Local date')[['D', 'NG: SUN', 'NG: WND']].sum().reset_index()

In [64]:
cal_bydate

,Local date,D,NG: SUN,NG: WND
0,2019-01-01,646739,-107.0,4259.0
1,2019-01-02,713041,163.0,8850.0
2,2019-01-03,723967,-18.0,9940.0
3,2019-01-04,715678,-225.0,7282.0
4,2019-01-05,706611,992.0,9466.0
...,...,...,...,...
2142,2024-11-12,700580,-247.0,10039.0
2143,2024-11-13,713495,-207.0,6196.0
2144,2024-11-14,700962,547.0,971.0
2145,2024-11-15,707880,736.0,0.0


In [29]:
cal_bydate = cal_bydate.groupby(cal_bydate['Local date']).sum()

In [66]:
cal_bydate

,Local date,D,NG: SUN,NG: WND
0,2019-01-01,646739,-107.0,4259.0
1,2019-01-02,713041,163.0,8850.0
2,2019-01-03,723967,-18.0,9940.0
3,2019-01-04,715678,-225.0,7282.0
4,2019-01-05,706611,992.0,9466.0
...,...,...,...,...
2142,2024-11-12,700580,-247.0,10039.0
2143,2024-11-13,713495,-207.0,6196.0
2144,2024-11-14,700962,547.0,971.0
2145,2024-11-15,707880,736.0,0.0


In [79]:
# Ensure the 'Local date' in energy data is in date format (strip time if necessary)
cal_bydate['Local date'] = pd.to_datetime(cal_bydate['Local date']).dt.date

# Ensure the 'DATE' in temperature data is in date format
all_summer_data['DATE'] = pd.to_datetime(all_summer_data['DATE']).dt.date

# Merge the datasets on the date columns
# Since we want to retain all rows in filtered_data and append the temperature data, use a 'left' join
combined_data = pd.merge(
    cal_bydate,
    all_summer_data,
    left_on='Local date',
    right_on='DATE',
    how='right'
)

# Drop the duplicate 'DATE' column from all_summer_data if needed
combined_data.drop(columns=['DATE'], inplace=True)

# Display the combined dataset
combined_data

,Local date,D,NG: SUN,NG: WND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
0,2024-06-01,683990,2174.0,0.0,NaN,NaN,NaN,72.0,85.0,59.0
1,2024-06-02,648971,2477.0,0.0,NaN,NaN,NaN,69.0,84.0,56.0
2,2024-06-03,719760,3090.0,0.0,NaN,NaN,NaN,65.0,74.0,58.0
3,2024-06-04,785081,1117.0,0.0,NaN,NaN,NaN,77.0,94.0,58.0
4,2024-06-05,865177,1257.0,3948.0,NaN,NaN,NaN,83.0,99.0,69.0
...,...,...,...,...,...,...,...,...,...,...
446747,2021-09-11,888781,-258.0,4647.0,NaN,NaN,NaN,68.0,73.0,62.0
446748,2021-09-12,844668,-309.0,5665.0,NaN,NaN,NaN,64.0,71.0,58.0
446749,2021-09-13,895106,-262.0,2671.0,NaN,NaN,NaN,59.0,64.0,55.0
446750,2021-09-14,864479,-427.0,4281.0,NaN,NaN,NaN,61.0,68.0,53.0


In [90]:
combined_data['Local date'] = pd.to_datetime(combined_data['Local date'], format='%d%b%Y')
combined_data['month'] = combined_data['Local date'].dt.month
combined_data['year'] = combined_data['Local date'].dt.year
combined_data['dotw'] = combined_data['Local date'].dt.dayofweek

In [92]:
combined_data['PRCP'] = combined_data['PRCP'].fillna(0)
combined_data['SNOW'] = combined_data['SNOW'].fillna(0)
combined_data['SNWD'] = combined_data['SNWD'].fillna(0)

combined_data['TAVG'] = combined_data.groupby('month')['TAVG'].transform(lambda x: x.fillna(x.mean()))
combined_data['TMAX'] = combined_data.groupby('month')['TMAX'].transform(lambda x: x.fillna(x.mean()))
combined_data['TMIN'] = combined_data.groupby('month')['TMIN'].transform(lambda x: x.fillna(x.mean()))

In [178]:
# Assuming 'combined_data' is your DataFrame and 'TMAX' is the column for maximum temperatures

# Step 1: Calculate the 95th percentile of TMAX
tmax_95th_percentile = combined_data['TAVG'].quantile(0.95)

# Step 2: Filter the DataFrame for days with TMAX in the 5th percentile highest temperatures
combined_data['Extreme'] = combined_data['TAVG'] >= tmax_95th_percentile

# Display the filtered DataFrame
combined_data

,Local date,D,NG: SUN,NG: WND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,month,year,dotw,Extreme,pscore
0,2024-06-01,683990,2174.0,0.0,0.0,0.0,0.0,72.0,85.0,59.0,6,2024,5,False,0.176541
1,2024-06-02,648971,2477.0,0.0,0.0,0.0,0.0,69.0,84.0,56.0,6,2024,6,False,0.178370
2,2024-06-03,719760,3090.0,0.0,0.0,0.0,0.0,65.0,74.0,58.0,6,2024,0,False,0.167615
3,2024-06-04,785081,1117.0,0.0,0.0,0.0,0.0,77.0,94.0,58.0,6,2024,1,False,0.169371
4,2024-06-05,865177,1257.0,3948.0,0.0,0.0,0.0,83.0,99.0,69.0,6,2024,2,True,0.171141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446747,2021-09-11,888781,-258.0,4647.0,0.0,0.0,0.0,68.0,73.0,62.0,9,2021,5,False,0.223688
446748,2021-09-12,844668,-309.0,5665.0,0.0,0.0,0.0,64.0,71.0,58.0,9,2021,6,False,0.225872
446749,2021-09-13,895106,-262.0,2671.0,0.0,0.0,0.0,59.0,64.0,55.0,9,2021,0,False,0.212994
446750,2021-09-14,864479,-427.0,4281.0,0.0,0.0,0.0,61.0,68.0,53.0,9,2021,1,False,0.215103


In [180]:
X = combined_data[['dotw', 'year', 'PRCP', 'SNOW', 'SNWD']]
y = combined_data['Extreme'] 

In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
model = LogisticRegression()

X = combined_data[['dotw', 'month']]
y = combined_data['Extreme'] 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression(max_iter=1000)
model.fit(X_scaled, y)

LogisticRegression(max_iter=1000)

In [184]:
propensity_scores = model.predict_proba(X)[:, 1]
propensity_df = pd.DataFrame(propensity_scores, columns=['Propensity Score'])
propensity_df

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,Propensity Score
0,0.104983
1,0.107947
2,0.091218
3,0.093834
4,0.096518
...,...
446747,0.136495
446748,0.140210
446749,0.119149
446750,0.122458


In [186]:
combined_data['pscore'] = propensity_scores

In [188]:
narrowed_prop = combined_data[(combined_data['pscore'] > 0.1) & (combined_data['pscore'] < 0.9)]
n = len(narrowed_prop)
treated_rows_cleaned = narrowed_prop.loc[narrowed_prop['Extreme'] == 1].copy()
untreated_rows_cleaned = narrowed_prop.loc[narrowed_prop['Extreme'] == 0].copy()
treated_rows_cleaned['Weighted Demand'] = treated_rows_cleaned['D'] * (1 / treated_rows_cleaned['pscore'])
untreated_rows_cleaned['Weighted Demand'] = untreated_rows_cleaned['D'] * (1 / (1 - untreated_rows_cleaned['pscore']))
trimmed_ipw_estimate = ((1 / n) * treated_rows_cleaned['Weighted Demand'].sum()) - ((1 / n) * untreated_rows_cleaned['Weighted Demand'].sum())
trimmed_ipw_estimate

-473538.7873792472